In [1]:
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from joblib import dump
from joblib import load


In [10]:
# データの読み込み
train_data = pd.read_csv('../data_processed/train_data_processed.csv')
test_data = pd.read_csv('../data_processed/test_data_processed.csv')
submit_data = pd.read_csv('../data/test.csv')

X = train_data.drop(['attendance'], axis=1)
y = train_data['attendance']

# 学習データと評価データに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 評価データをDMatrixに変換
dtest = xgb.DMatrix(X_test)

In [6]:
# モデルを読み込む
XGBoost = xgb.Booster()
XGBoost.load_model('../models/best_model(XGBoost)')

In [11]:
# 評価データで予測
predictions = XGBoost.predict(dtest)

# RMSEを計算
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

# 評価用データをDMatrixに変換
dsubmit = xgb.DMatrix(test_data)

# 評価用データで予測
test_predictions = XGBoost.predict(dsubmit)

# 提出用データフレームを作成
submission = pd.DataFrame({
    'id': submit_data['id'],
    'attendance': test_predictions
})

# ヘッダーを含まずにCSVファイルとして保存（'Deloitte Analytics/submission'フォルダに保存）
submission.to_csv('../submission/9XGBoost_regression.csv', index=False, header=False)

RMSE: 4112.9794823963775
